In [198]:
from typing import Dict, List, Optional

import plotly.graph_objects as go


def plot_investment_tragectory(
    assest_return: Dict[str, float],
    assest_allocation: Dict[str, float],
    investemnt_amount: float,
    years_projection: int,
    yearly_spending: float,
    yearly_earning: float,
    tax_rate: float,
    capital_gain_tax_rate: float,
    start_age: int,
    inflation_rate: Optional[
        float
    ] = 0.0,  # Optional parameter for inflation rate with default value of 0.0
    inflation_adjusted_cashflow: Optional[
        bool
    ] = False,  # Flag to make spending and earning inflation adjusted
    years_with_earnings: Optional[
        int
    ] = None,  # Number of years with earnings, after which earnings stop
) -> None:
    # If years_with_earnings is not specified, assume earnings for all years
    if years_with_earnings is None:
        years_with_earnings = years_projection

    yearly_projections_nominal = [investemnt_amount]  # Track nominal values

    for year in range(1, years_projection + 1):
        total_return = 0
        total_interest = 0  # Track interest earned this year

        for asset, allocation in assest_allocation.items():
            asset_amount = yearly_projections_nominal[-1] * allocation
            asset_return = assest_return.get(asset, 0)
            interest = asset_amount * (
                asset_return / 100.0
            )  # Calculate interest for this asset
            total_interest += interest  # Accumulate interest
            total_return += asset_amount * (1 + (asset_return / 100.0))

        # Determine if earnings apply this year
        current_yearly_earning = yearly_earning if year <= years_with_earnings else 0

        # Adjust spending and earning for inflation if flag is set
        if inflation_adjusted_cashflow:
            inflation_factor = (1 + inflation_rate / 100.0) ** year
            adjusted_yearly_spending = yearly_spending * inflation_factor
            adjusted_yearly_earning = current_yearly_earning * inflation_factor
        else:
            adjusted_yearly_spending = yearly_spending
            adjusted_yearly_earning = current_yearly_earning

        yearly_earning_after_tax = adjusted_yearly_earning * (1 - tax_rate)
        yearly_net_inflow = yearly_earning_after_tax - adjusted_yearly_spending

        if yearly_net_inflow < 0:  # If net inflow is negative, apply capital gains tax
            yearly_net_inflow *= 1 + capital_gain_tax_rate
        new_total_nominal = total_return + yearly_net_inflow
        yearly_projections_nominal.append(new_total_nominal)

    # Find the year when nominal projection hits zero
    zero_year = None
    for i in range(len(yearly_projections_nominal)):
        if yearly_projections_nominal[i] <= 0:
            zero_year = i
            break

    # Apply inflation adjustment to convert nominal to real values
    inflation_factor = 1 + inflation_rate / 100.0
    yearly_projections = [
        yearly_projections_nominal[i] / (inflation_factor**i)
        for i in range(years_projection + 1)
    ]
    inflation_adjustment = [
        investemnt_amount / (inflation_factor**i) for i in range(years_projection + 1)
    ]

    # Create age-based x-axis
    ages = [start_age + year for year in range(years_projection + 1)]

    # Interactive plot with Plotly
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=ages,
            y=yearly_projections,
            mode="lines+markers",
            name="Total Investment Value",
        )
    )

    # Add vertical red line where nominal hits zero
    if zero_year is not None:
        fig.add_vline(
            x=start_age + zero_year,
            line_width=2,
            line_dash="dash",
            line_color="red",
            annotation_text=f"Zero at age {start_age + zero_year}",
            annotation_position="top",
        )

    fig.update_layout(
        title="Investment Trajectory Over Time (Inflation Adjusted)",
        xaxis_title="Age",
        yaxis_title="Amount (in thousands)",
        yaxis_tickformat=",.2f",
        xaxis=dict(
            tickmode="linear",
            tick0=start_age,
            dtick=1,
            range=[start_age, start_age + years_projection],
        ),
        yaxis=dict(
            zeroline=True,
            zerolinecolor='blue',
            range=[min(min(yearly_projections), 0) - 10, max(yearly_projections) + 10],
        ),
        template="plotly_white",
    )

    fig.show()

    # Nominal values plot
    fig_nominal = go.Figure()
    fig_nominal.add_trace(
        go.Scatter(
            x=ages,
            y=yearly_projections_nominal,
            mode="lines+markers",
            name="Total Investment Value (Nominal)",
        )
    )

    # Add vertical red line where nominal hits zero
    if zero_year is not None:
        fig_nominal.add_vline(
            x=start_age + zero_year,
            line_width=2,
            line_dash="dash",
            line_color="red",
            annotation_text=f"Zero at age {start_age + zero_year}",
            annotation_position="top",
        )

    fig_nominal.update_layout(
        title="Investment Trajectory Over Time (Nominal Values)",
        xaxis_title="Age",
        yaxis_title="Amount (in thousands)",
        yaxis_tickformat=",.2f",
        xaxis=dict(
            tickmode="linear",
            tick0=start_age,
            dtick=1,
            range=[start_age, start_age + years_projection],
        ),
        yaxis=dict(
            zeroline=True,
            zerolinecolor='blue',
            range=[min(min(yearly_projections_nominal), 0) - 10, max(yearly_projections_nominal) + 10],
        ),
        template="plotly_white",
    )

    fig_nominal.show()

In [199]:
plot_investment_tragectory(
    assest_return= {
        'bonds': 4,
        'stocks': 9
    },
    assest_allocation= {
        'bonds': 0.2,
        'stocks': 0.8
    },
    investemnt_amount = 1_500_000,
    years_projection = 55, 
    yearly_spending = 12 * 14_000, 
    yearly_earning = 50_000,
    tax_rate = 0.2,
    capital_gain_tax_rate = 0.2, 
    years_with_earnings = 20,
    inflation_rate = 2.0,
    inflation_adjusted_cashflow = True, 
    start_age = 40, 
)

In [166]:
plot_investment_tragectory(
    assest_return= {
        'bonds': 4,
        'stocks': 9
    },
    assest_allocation= {
        'bonds': 0.2,
        'stocks': 0.8
    },
    investemnt_amount = 1_500_000,
    years_projection = 55, 
    yearly_spending = 12 * 14_000, 
    yearly_earning = 60_000,
    tax_rate = 0.2,
    capital_gain_tax_rate = 0.2, 
    years_with_earnings = 20,
    inflation_rate = 2.0,
    inflation_adjusted_cashflow = True, 
    start_age = 40, 
)

In [195]:
# barista + spending 14k monthly (current)
plot_investment_tragectory(
    assest_return= {
        'bonds': 4,
        'stocks': 9
    },
    assest_allocation= {
        'bonds': 0.2,
        'stocks': 0.8
    },
    investemnt_amount = 1_500_000,
    years_projection = 55, 
    yearly_spending = 12 * (12000), 
    yearly_earning = 80_000, # barista for two people
    tax_rate = 0.25,
    capital_gain_tax_rate = 0.2, 
    years_with_earnings = 20,
    inflation_rate = 2.0,
    inflation_adjusted_cashflow = True, 
    start_age = 40, 
)

In [179]:
# barista + spending 10k monthly 
plot_investment_tragectory(
    assest_return= {
        'bonds': 4,
        'stocks': 9
    },
    assest_allocation= {
        'bonds': 0.2,
        'stocks': 0.8
    },
    investemnt_amount = 1_500_000,
    years_projection = 55, 
    yearly_spending = 12 * (10000), 
    yearly_earning = 60_000, # barista for two people
    tax_rate = 0.25,
    capital_gain_tax_rate = 0.2, 
    years_with_earnings = 20,
    inflation_rate = 2.0,
    inflation_adjusted_cashflow = True, 
    start_age = 40, 
)

In [180]:
# coasting + spending currently
plot_investment_tragectory(
    assest_return= {
        'bonds': 4,
        'stocks': 9
    },
    assest_allocation= {
        'bonds': 0.2,
        'stocks': 0.8
    },
    investemnt_amount = 1_500_000,
    years_projection = 55, 
    yearly_spending = 12 * (14000), 
    yearly_earning = 100_000, # barista for two people
    tax_rate = 0.25,
    capital_gain_tax_rate = 0.2, 
    years_with_earnings = 20,
    inflation_rate = 2.0,
    inflation_adjusted_cashflow = True, 
    start_age = 40, 
)

In [181]:
# coasting + spending 10k
plot_investment_tragectory(
    assest_return= {
        'bonds': 4,
        'stocks': 9
    },
    assest_allocation= {
        'bonds': 0.2,
        'stocks': 0.8
    },
    investemnt_amount = 1_500_000,
    years_projection = 55, 
    yearly_spending = 12 * (10000), 
    yearly_earning = 100_000, # barista for two people
    tax_rate = 0.25,
    capital_gain_tax_rate = 0.2, 
    years_with_earnings = 20,
    inflation_rate = 2.0,
    inflation_adjusted_cashflow = True, 
    start_age = 40, 
)

In [187]:
# coasting + spending 14k
plot_investment_tragectory(
    assest_return= {
        'bonds': 4,
        'stocks': 9
    },
    assest_allocation= {
        'bonds': 0.2,
        'stocks': 0.8
    },
    investemnt_amount = 2_000_000,
    years_projection = 55, 
    yearly_spending = 12 * (14000), 
    yearly_earning = 150_000, # barista for two people
    tax_rate = 0.25,
    capital_gain_tax_rate = 0.2, 
    years_with_earnings = 10,
    inflation_rate = 2.0,
    inflation_adjusted_cashflow = True, 
    start_age = 40, 
)